In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from IPython.core.display import display, HTML, clear_output
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.stats import sem

import os
import sys

import matplotlib.pyplot as plt
import matplotlib.ticker
from venn import venn, pseudovenn
import seaborn as sns
from matplotlib.collections import PathCollection

from Bio import SeqIO, SeqUtils
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from addict import Dict
import json


In [4]:
sys.path.append("../")
import plotting as my_plot

sys.path.append("./mean_field_elect/")
import proteomics_functions as prot_fun
import base_classes
import morbidelli as m

# Load data

In [6]:
dfs = Dict()
excel_obj = pd.ExcelFile('./data/Chase DDA 20220830.xlsx')
dfs.cq.g12 = excel_obj.parse('CaptoQ FT-1G12')
dfs.cq.eluate = excel_obj.parse('CaptoQ FT-Eluate')
dfs.hq.g12 = excel_obj.parse('Poros50HQ FT-1G12')
dfs.hq.eluate = excel_obj.parse('Poros50HQ FT-Eluate')
dfs.xq.g12 = excel_obj.parse('PorosXQ FT-1G12')
dfs.xq.eluate = excel_obj.parse('PorosXQ FT-Eluate')

# Remove non-CHO and reversed (decoy) proteins

In [7]:
def remove_non_CHO_and_reversed(df):
    df = df[df['Name'].str.contains('Cricetulus griseus')]
    df_rev = df[df['Accession'].str.contains('RRRRR')]
    df = df[~df['Accession'].str.contains('RRRRR')]
    df.reset_index(inplace=True, drop=True)
    df_rev.reset_index(inplace=True, drop=True)
    return df, df_rev

In [8]:
dfs_rev = Dict()

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        dfs[resin][frac], df_rev = remove_non_CHO_and_reversed(dfs[resin][frac])
        if not df_rev.empty:
            dfs_rev[resin][frac] = df_rev

## What confidence criteria were given to reversed proteins?

In [9]:
for resin in dfs_rev.keys():
    for frac in dfs_rev[resin].keys():
        df = dfs_rev[resin][frac]
        print(resin, frac, '\t', df.at[0, 'Unused'], '\t', df.at[0, 'Peptides(95%)'])

hq eluate 	 2.0 	 1
xq eluate 	 0.0 	 1


# Select HCPs with $\geq 2$ peptides

In [10]:
for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        df.drop(df[df['Peptides(95%)'] == 1].index, inplace=True)
        df.reset_index(inplace=True, drop=True)

# Deal with the problem of $Unused \; score = 0$

Problem:  among multiple possible HCPs, only one needs to be selected, but this can lead to the appearance of different HCPs in different samples that should truly be treated as the same HCP.

I plan to try to minimize the total number of HCPs in the union by selecting among HCP possibilities based on some ranking (eg. most confident identifications elswhere, etc.).

In [13]:
accn_groups = []
single_ids = []

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        if len(df) > 0:
            n_vals = list(set(df.N))
            n_vals.sort()
            for n in n_vals:
                df_prot = df[df.N == n]
                accn_groups.append(list(df_prot.Accession))
                if len(df_prot) == 1:
                    single_ids.append([resin, frac, df_prot.iloc[0]['Accession'], df_prot.iloc[0]['Unused'], df_prot.iloc[0]['Peptides(95%)']])
                    
unique_accn_groups = list(set(map(tuple, accn_groups)))
df_singles = pd.DataFrame(single_ids, columns=['resin', 'frac', 'accn', 'unused', 'peptides'])

In [14]:
# Get frequency counts for HCPs
hcp_freq = {}

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        for a in df.Accession:
            if a in hcp_freq.keys():
                hcp_freq[a] += 1
            else:
                hcp_freq[a] = 1

In [15]:
# Select "winner" HCPs
# 1) Are there multiple options? If not, the selection is unnecessary.
# 2) If there are multiple options, narrow it down by taking the HCPs with the highest number of peptides. Then, are any of the HCPs found elsewhere as a singlet? 
#    If so, select the singlet that was identified with the greatest confidence (based on number of peptides, then the unused score)
# 3) If none of the degenerate HCPs are ever found as singlets, check that I haven't already selected any of the HCPs (from 4 and 5 below).
#    If previous selections have been made, take the one that has the higher frequency of previous selection, 
#    and if they all have the same frequency of previous selection, take the top one from the list of ordered accn ids.
# 4) If none of the HCPs have been previously selected, does any one of the HCPs appear at a higher frequency? If so, select that one.
# 5) If not, take the top entry in the list of ordered accn ids. 

already_selected = {}

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        df['selection'] = False
        if len(df) > 0:
            n_vals = list(set(df.N))
            n_vals.sort()
            for n in n_vals:
                df_prot = df[df.N == n]
                if len(df_prot) == 1:
                    index = df_prot.index[0]
                    df.at[index, 'selection'] = True
                else:
                    df_prot = df_prot[df_prot['Peptides(95%)'] == df_prot['Peptides(95%)'].max()]
                    df_single_test = df_singles[df_singles.accn.isin(list(df_prot.Accession))].copy()
                    if len(df_single_test) > 0:
                        df_single_test.sort_values(by=['peptides', 'unused'], inplace=True, ignore_index=True, ascending=False)
                        accn_selection = df_single_test.at[0, 'accn']
                        index = df_prot.loc[df_prot.Accession == accn_selection].index[0]
                        df.at[index, 'selection'] = True
                    else:
                        df_already = df_prot[df_prot.Accession.isin(already_selected.keys())].copy()
                        if len(df_already) == 0:
                            temp = []
                            for a in df_prot.Accession:
                                temp.append([a, hcp_freq[a]])
                            df_freq = pd.DataFrame(temp, columns=['accn', 'freq'])
                            df_freq.sort_values(by=['freq', 'accn'], inplace=True, ignore_index=True, ascending=False)
                            accn_selection = df_freq.at[0, 'accn']
                            index = df_prot.loc[df_prot.Accession == accn_selection].index[0]
                            df.at[index, 'selection'] = True
                            already_selected[accn_selection] = 1
                        else:
                            df_b = df_already.copy()
                            for i, cont in df_already.iterrows():
                                df_already.at[i, 'freq'] = already_selected[cont.Accession]
                            df_already.sort_values(by=['freq', 'Accession'], inplace=True, ignore_index=True, ascending=False)
                            accn_selection = df_already.at[0, 'Accession']
                            index = df_prot.loc[df_prot.Accession == accn_selection].index[0]
                            df.at[index, 'selection'] = True
                            already_selected[accn_selection] += 1                         

In [16]:
# Check
for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        assert df.selection.sum() - len(list(set(df.N))) == 0
        if len(df) > 0:
            n_vals = list(set(df.N))
            n_vals.sort()
            for n in n_vals:
                df_prot = df[df.N == n]
                assert df_prot.selection.sum() == 1            

# Get set of all proteins and a master dataframe with their locations

In [18]:
names = {}

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        df = df[df.selection]
        for i, cont in df.iterrows():
            names[cont['Accession']] = cont['Name']
                
df_master_1 = pd.DataFrame.from_dict(names, orient='index')
df_master_1.reset_index(inplace=True)
df_master_1.columns = ['accession', 'name']

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        df = df[df.selection]
        for i, cont in df_master_1.iterrows():
            df_master_1.at[i, f'{resin}_{frac}'] = cont.accession in list(df['Accession'])

In [19]:
# For reference - if I had just selected the first species in each degenerate case

names = {}

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        df = df[df.Unused > 0]
        for i, cont in df.iterrows():
            names[cont['Accession']] = cont['Name']
                
df_master_dummy = pd.DataFrame.from_dict(names, orient='index')
df_master_dummy.reset_index(inplace=True)
df_master_dummy.columns = ['accession', 'name']

for resin in dfs.keys():
    for frac in dfs[resin].keys():
        df = dfs[resin][frac]
        df = df[df.Unused > 0]
        for i, cont in df_master_dummy.iterrows():
            df_master_dummy.at[i, f'{resin}_{frac}'] = cont.accession in list(df['Accession'])
                
len(df_master_dummy), len(df_master_1), (len(df_master_dummy) - len(df_master_1))/len(df_master_dummy)

(1349, 1271, 0.05782060785767235)

# Add sequences, pI values, and masses to the dataframe

In [23]:
# # Get new html links to look up sequences

# html = ''
# cnt = 0

# for i, a in enumerate(df_master_1.accession):
#     if i % 200 == 0:
#         print(html[:-1], '\n'*2)
#         html = 'https://www.ncbi.nlm.nih.gov/protein/'        
#     html += a + ','
#     cnt += 1
    
# print(html[:-1], '\n'*2)

In [24]:
# My substitution rules for uncertain amino acids
my_sub_rules = {'B':'D', 'Z':'E', 'X':'A', 'J':'L'}

# Get sequence dictionary {accession:sequence_object}
sequences = {}
subbed_ids = []

for r in SeqIO.parse("./data/sequences_dda.fasta", "fasta"):
    for aa in my_sub_rules.keys(): # uncertain amino acids
        if aa in r.seq:
            r.seq = r.seq.replace(aa, my_sub_rules[aa])
            subbed_ids.append(r.id)
    sequences[r.id] = r.seq

In [27]:
missing = []
for i, cont in df_master_1.iterrows():
    if cont.accession not in list(sequences.keys()):
        missing.append(cont.accession)
assert len(missing) == 0

In [28]:
for accn in df_master_1.accession:
    assert accn in sequences.keys()

In [29]:
# Get pI and mass dictionaries {accession:pI/mass}
pI_vals = {}
masses = {}

for p_id, seq in sequences.items():
    pI, is_solved = prot_fun.get_pI(seq)
    assert is_solved
    pI_vals[p_id] = pI
    masses[p_id] = SeqUtils.molecular_weight(seq, seq_type='protein')

/home/chase/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [30]:
# Add sequences, pI values, and masses to df_master_1
for i, cont in df_master_1.iterrows():
    df_master_1.at[i, 'sequence'] = str(sequences[cont.accession])
    df_master_1.at[i, 'pI'] = pI_vals[cont.accession]
    df_master_1.at[i, 'mass'] = masses[cont.accession]

In [33]:
# Get other biophysical property dictionaries (assuming pH 7.0)
net_charges, net_neg_charges, net_pos_charges, charge_densities, charge_densities_neg, charge_densities_pos = {}, {}, {}, {}, {}, {}

for p_id, seq in sequences.items():
    net_charge, net_neg_charge, net_pos_charge, charge_dens, charge_dens_neg, charge_dens_pos = prot_fun.get_charge(pH=7.0, seq=seq, charge_contributions=True)
    net_charges[p_id] = net_charge
    net_neg_charges[p_id] = net_neg_charge
    net_pos_charges[p_id] = net_pos_charge
    charge_densities[p_id] = charge_dens
    charge_densities_neg[p_id] = charge_dens_neg
    charge_densities_pos[p_id] = charge_dens_pos

In [34]:
# Add these biophysical properties to df_master_1
for i, cont in df_master_1.iterrows():
    df_master_1.at[i, 'net_charge'] = net_charges[cont.accession]
    df_master_1.at[i, 'net_charge_neg'] = net_neg_charges[cont.accession]
    df_master_1.at[i, 'net_charge_pos'] = net_pos_charges[cont.accession]
    df_master_1.at[i, 'charge_dens_C_m2'] = charge_densities[cont.accession]
    df_master_1.at[i, 'charge_dens_neg_C_m2'] = charge_densities_neg[cont.accession]
    df_master_1.at[i, 'charge_dens_pos_C_m2'] = charge_densities_pos[cont.accession]

In [35]:
# Check for duplicate sequences (i.e. degeneracy of accession numbers corresponding to a single sequence)
sequence_dict = {}
for i, s in enumerate(df_master_1.sequence):
    if s in sequence_dict.keys():
        sequence_dict[s].append(i)
    else:
        sequence_dict[s] = [i]
        
dup_seq = [s for s, indeces in sequence_dict.items() if len(indeces) > 1]
dup_seq_indeces = [indeces for n, indeces in sequence_dict.items() if len(indeces) > 1]

print(dup_seq, dup_seq_indeces)

[] []


In [36]:
# Get cysteine content
for i, cont in df_master_1.iterrows():
    x = ProteinAnalysis(str(sequences[cont.accession]))
    df_master_1.at[i, 'cysteine_cont_percent'] = x.get_amino_acids_percent()['C'] * 100
    df_master_1.at[i, 'cysteine_num'] = x.count_amino_acids()['C']

In [37]:
# Drop sequences for readability
try:
    df_master_1.drop(columns=['sequence'], inplace=True)
except:
    pass

# Combine entries with identically repeated names

In [45]:
sample_columns = ['cq_g12', 'cq_eluate', 'hq_g12', 'hq_eluate', 'xq_g12', 'xq_eluate']

In [38]:
hcp_names = {}
for i, n in enumerate(df_master_1.name):
    if n in hcp_names.keys():
        hcp_names[n].append(i)
    else:
        hcp_names[n] = [i]
        
dup_names = [n for n, indeces in hcp_names.items() if len(indeces) > 1]
dup_indeces = [indeces for n, indeces in hcp_names.items() if len(indeces) > 1]
dup_indeces_flat = [item for sublist in dup_indeces for item in sublist]

n_dup_max = max([len(sublist) for sublist in dup_indeces])

In [46]:
for indeces in dup_indeces:
    df_temp = df_master_1.iloc[indeces]
    for sample in sample_columns:
        if df_temp[sample].sum() > 1:
            print(indeces, '\t', sample,  '\t', df_temp.at[indeces[0], 'name'])

[185, 1006] 	 hq_eluate 	 beta-hexosaminidase subunit beta isoform X2 [Cricetulus griseus]
[185, 1006] 	 xq_eluate 	 beta-hexosaminidase subunit beta isoform X2 [Cricetulus griseus]


In [43]:
# df_master_1.iloc[dup_indeces_flat]

In [47]:
unique_names = df_master_1.iloc[~df_master_1.index.isin(dup_indeces_flat)].copy()
unique_names.reset_index(inplace=True, drop=True)
unique_names.rename(columns={'accession':'accession_0', 'pI':'pI_0', 'mass':'mass_0', 'net_charge':'net_charge_0', 
                             'net_charge_neg':'net_charge_neg_0', 'net_charge_pos':'net_charge_pos_0',
                             'charge_dens_C_m2':'charge_dens_C_m2_0', 'charge_dens_neg_C_m2':'charge_dens_neg_C_m2_0', 
                             'charge_dens_pos_C_m2':'charge_dens_pos_C_m2_0', 'cysteine_cont_percent':'cysteine_cont_percent_0',
                             'cysteine_num':'cysteine_num_0'}, inplace=True)

for n in dup_names:
    df = df_master_1.iloc[hcp_names[n]].copy()
    df.reset_index(inplace=True, drop=True)
    combined_entry = {}
    
    for i, cont in df.iterrows():
        for column in ['accession', 'pI', 'mass', 'net_charge', 'net_charge_neg', 'net_charge_pos',
                       'charge_dens_C_m2', 'charge_dens_neg_C_m2', 'charge_dens_pos_C_m2',
                       'cysteine_cont_percent', 'cysteine_num']:
            combined_entry[f'{column}_{i}'] = [cont[column]]
        
        if i == 0:
            combined_entry['name'] = [cont['name']]
    
    for sample in sample_columns:
        combined_entry[sample] = [True in list(df[sample])]
    
    df_temp = pd.DataFrame(combined_entry)
    unique_names = pd.concat([unique_names, df_temp], ignore_index=True)

In [48]:
pI_columns = []
mass_columns = []
net_charge_columns = []
net_charge_neg_columns = []
net_charge_pos_columns = []
charge_dens_C_m2_columns = []
charge_dens_neg_C_m2_columns = []
charge_dens_pos_C_m2_columns = []
cysteine_cont_percent_columns = []
cysteine_num_columns = []

for i in range(n_dup_max):
    pI_columns.append(f'pI_{i}')
    mass_columns.append(f'mass_{i}')
    net_charge_columns.append(f'net_charge_{i}')
    net_charge_neg_columns.append(f'net_charge_neg_{i}')
    net_charge_pos_columns.append(f'net_charge_pos_{i}')
    charge_dens_C_m2_columns.append(f'charge_dens_C_m2_{i}')
    charge_dens_neg_C_m2_columns.append(f'charge_dens_neg_C_m2_{i}')
    charge_dens_pos_C_m2_columns.append(f'charge_dens_pos_C_m2_{i}')
    cysteine_cont_percent_columns.append(f'cysteine_cont_percent_{i}')
    cysteine_num_columns.append(f'cysteine_num_{i}')
    
unique_names['pI_mean'] = unique_names[pI_columns].mean(axis=1)
unique_names['mass_mean'] = unique_names[mass_columns].mean(axis=1)
unique_names['net_charge_mean'] = unique_names[net_charge_columns].mean(axis=1)
unique_names['net_charge_neg_mean'] = unique_names[net_charge_neg_columns].mean(axis=1)
unique_names['net_charge_pos_mean'] = unique_names[net_charge_pos_columns].mean(axis=1)
unique_names['charge_dens_C_m2_mean'] = unique_names[charge_dens_C_m2_columns].mean(axis=1)
unique_names['charge_dens_neg_C_m2_mean'] = unique_names[charge_dens_neg_C_m2_columns].mean(axis=1)
unique_names['charge_dens_pos_C_m2_mean'] = unique_names[charge_dens_pos_C_m2_columns].mean(axis=1)
unique_names['cysteine_cont_percent_mean'] = unique_names[cysteine_cont_percent_columns].mean(axis=1)
unique_names['cysteine_num_mean'] = unique_names[cysteine_num_columns].mean(axis=1)

# Save master dataframe

In [49]:
df_master = unique_names
for i, cont in df_master.iterrows():
    text = cont['name'].replace(' [Cricetulus griseus]', '')
    df_master.at[i, 'desc_lower'] = text.lower()

In [50]:
for i, cont in df_master.iterrows():
    desc_lower_2 = cont.desc_lower[:]
    desc_lower_2 = desc_lower_2.replace('-', ' ')
    desc_lower_2 = desc_lower_2.replace('_', ' ')
    desc_lower_2 = desc_lower_2.replace(',', '')
    df_master.at[i, 'desc_lower_2'] = desc_lower_2

In [55]:
# df_master.to_csv('./generated_tables/df_master_dda_with_properties.csv', index=False)

In [54]:
df_master_pared_down = df_master.copy()
df_master_pared_down.rename(columns={'accession_0':'accn'}, inplace=True)
df_master_pared_down.drop(columns=['pI_0', 'mass_0', 'net_charge_0', 'net_charge_neg_0', 'net_charge_pos_0',
                       'charge_dens_C_m2_0', 'charge_dens_neg_C_m2_0',
                       'charge_dens_pos_C_m2_0', 'accession_1', 'pI_1', 'mass_1',
                       'net_charge_1', 'net_charge_neg_1', 'net_charge_pos_1',
                       'charge_dens_C_m2_1', 'charge_dens_neg_C_m2_1',
                       'charge_dens_pos_C_m2_1', 'cysteine_cont_percent_0', 'cysteine_num_0',
                       'cysteine_cont_percent_1', 'cysteine_num_1'], inplace=True)
# df_master_pared_down.to_csv('./generated_tables/df_master_dda_with_mean_properties.csv', index=False)

In [60]:
df_master_bare = df_master_pared_down.copy()
df_master_bare.rename(columns={'pI_mean':'pI', 'mass_mean':'mass'}, inplace=True)
df_master_bare.drop(columns=['net_charge_mean', 'net_charge_neg_mean', 'net_charge_pos_mean', 'charge_dens_C_m2_mean',
                               'charge_dens_neg_C_m2_mean', 'charge_dens_pos_C_m2_mean',
                               'cysteine_cont_percent_mean', 'cysteine_num_mean', 'desc_lower'], inplace=True)
# df_master_bare.to_csv('./generated_tables/df_master_dda_with_bare_properties.csv', index=False)